In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
pd.set_option("display.max_columns", None, "display.width", 1000)

# EDA - projekt 1

**Mikołaj Spytek**
**Artur Żółkowski**

Zajmujemy się zbiorem danych, dotyczącym uczniów dwóch szkół średnich w Portugalii. Dane pochodzą zarówno z odpowiedzi udzielonych bezpośrednio przez uczniów (dane dotyczące sytuacji społecznej, ekonomicznej itd.) oraz z informacji przechowywyanych przez szkołę (dane dotyczące m. in. ocen).

Zmienną wyjaśnianą jest `G3` - ocena roczna. Pozostałe zmienne - w znaczej większości kategoryczne, to zmienne wyjaśniające. 

In [ ]:
r = requests.get('https://api.apispreadsheets.com/api/dataset/school-grades/')
data = r.json()
df = pd.DataFrame(data['data'])

df.head()

In [ ]:
df.info()

Patrząc na opis zbioru danych, możemy zauważyć, że w ramce nie występują wartości puste. Widać również, że cały zbiór danych składa się z 649 obserwacji. Dla zmiennych faktycznie numerycznych, warto spojrzeć na statystyki opisowe. Dla pozostałych kolumn, nie ma to zbyt dużego sensu, gdyż 

In [ ]:
df[["age", "failures", "absences", "G1","G2","G3"]].describe()

In [ ]:
df.hist(bins=20, figsize=(18,12))
plt.show()

Dane mamy z dwóch szkół, więc warto zbadać, czy klasy są zbalansowane.

In [ ]:
sns.countplot(data=df, x="school")
plt.show()

Niestety nie są - musimy to wziąć pod uwagę przy podziale na zbiór testowy i walidacyjny - ocenianie między szkołami może się różnic.

In [ ]:
df["G3"].hist(bins=20)
plt.title("Disribution of G3")
plt.show()

Widzimy, że oceny końcowe miałyby rozkład podobny do normalnego, gdyby nie osoby z oceną 0 - powoduje to jednak, że mamy do czynienia z ujemną skośnością.

Ciekawa rzecz pokazuje się nam, gdy spojrzymy na zmienną `G2` i jej korelację ze zmienną wyjaśnianą: 

In [ ]:
plt.figure(figsize=(18, 12))
sns.boxplot(data=df, x="G2", y="G3")
plt.show()

Jak zauważyli autorzy zbioru danych zmienne `G1` i `G2` są bardzo silnie skorelowane z naszą zmienną wyjaśnianą. Jednak użycie ich do predykowania `G3` jest mało przydatne, bo są to oceny z poprzednich semestrów. Dużo ciekawiej jest wytrenować model na pozostałych kolumnach. 

Poniżej przyjrzymy się kilku zmiennym, które według naszej intuicji mogłyby mieć większy wpływ na ocenę końcową.

In [ ]:
# różnica między tymi dwiema szkołami

sns.kdeplot(data=df.loc[df["school"]=="GP", "G3"], shade="True", legend=True, label = "GP")
sns.kdeplot(data=df.loc[df["school"]=="MS", "G3"], shade="True", legend=True, label = "MS")
plt.legend(title="School")

plt.show()

Już z pierwszego wykresu gęstości widzimy, że uczniowie szkoły `GP` uzyskują trochę lepsze oceny.

In [ ]:
# miasto/wieś

sns.kdeplot(data=df.loc[df["address"]=="U", "G3"], shade="True", legend=True, label = "Urban")
sns.kdeplot(data=df.loc[df["address"]=="R", "G3"], shade="True", legend=True, label = "Rural")
plt.legend(title="Address")
plt.show()

Na kolejnym wykresie widzimy, że uczniowie mieszkający w mieście mają trochę większą średnią ocenę, ale również nieco większą wariancję.

In [ ]:
plot = sns.countplot(x=df['address'])
plot.set_xlabel('Address')
plot.set_ylabel('Count')
plt.show()

Przy okazji sprawdziliśmy, jaka jest proporcja uczniów mieszkających w różnych miejscach - również nie mamy zbalansowanych kateogrii.

In [ ]:
# czy zamierzają się jeszcze uczyć

sns.kdeplot(data=df.loc[df["higher"]=="yes", "G3"], shade="True", legend=True, label = "yes")
sns.kdeplot(data=df.loc[df["higher"]=="no", "G3"], shade="True", legend=True, label = "no")
plt.legend(title="Higher education?")

plt.show()

Zmienną, na której bardzo widać różnice w końcowej ocenie jest chęć podjęcia dalszej edukacji po szkole średniej. 

In [ ]:
# dostęp do internetu

sns.kdeplot(data=df.loc[df["internet"]=="yes", "G3"], shade="True", legend=True, label = "yes")
sns.kdeplot(data=df.loc[df["internet"]=="no", "G3"], shade="True", legend=True, label = "no")
plt.legend(title="Internet?")
plt.show()

Dostęp do internetu również wpływa na rozkład ocen końcowych - nieznacznie, ale jednak.

In [ ]:
#niezaliczenia

sns.kdeplot(data=df.loc[df["failures"]==0, "G3"], shade="True", label="0")
sns.kdeplot(data=df.loc[df["failures"]>=1, "G3"], shade="True", label=">1")
plt.legend(title="liczba niezaliczen")

plt.show()

Znaczny wpływ ma zmienna określająca ile razy dany uczeń nie zaliczył kursu. Uczniowie, którzy nie zdali, mają średnio znacznie gorsze oceny.

Następne kilka wykresów jest według nas bardzo ciekawe. Zarówno płeć, jak i to, że dana osoba jest w związku nie ma dużego wpływu na rozkład ocen. Jednak jeśli popatrzymy na te zmienne razem to zmiany stają się bardziej zauważalne. Dodatkowo trend nie jest taki sam a odwraca się. 

In [ ]:
sns.kdeplot(df.loc[df['sex'] == 'F', 'G3'], label='Female', shade = True)
sns.kdeplot(df.loc[df['sex'] == 'M', 'G3'], label='Male', shade = True)
plt.title('Gender', fontsize = 14)
plt.legend(title="Gender")
plt.show()

In [ ]:
sns.kdeplot(df.loc[(df['romantic'] == 'yes') & (df['sex'] == 'F'), 'G3'], label='Relationship Female', shade = True)
sns.kdeplot(df.loc[(df['romantic'] == 'no') & (df['sex'] == 'F'), 'G3'], label='Single Female', shade = True)
plt.title('Relationship', fontsize = 14)
plt.legend()
plt.show()

In [ ]:
sns.kdeplot(df.loc[(df['romantic'] == 'yes') & (df['sex'] == 'M'), 'G3'], label='Relationship Male', shade = True)
sns.kdeplot(df.loc[(df['romantic'] == 'no') & (df['sex'] == 'M'), 'G3'], label='Single Male', shade = True)
plt.title('Relationship', fontsize = 14)
plt.legend()
plt.show()

Widać też, że edukacja rodziców w pewien sposób koreluje z oceną końcową dziecka.

In [ ]:
family_ed = df['Fedu'] + df['Medu'] 

plt.figure(figsize=(12,12))
b = sns.boxplot(x=family_ed,y=df['G3'])
b.set_xlabel('Parents education ')
b.set_ylabel('Final Grade')
b.set_ylim((0,21))
plt.show()

Na heatmapie sprawdziliśmy korelacje poszczególnych zmiennych. Zauważyliśmy jednak, że niektórych kolumn nie ma.

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(),annot=True, fmt=".2f", square=True)
plt.show()

Okazało się, że niektóre zmienne kategoryczne były zapisane w kolumnach tekstowych - musieliśmy je przekonwertować, aby pokazały się na heatmapie.

In [ ]:
df_all = df.copy()

for col in df_all.columns:
    if str(df_all[col].dtype) != "int64":
        df_all[col]=df_all[col].astype("category")
        df_all[col]=df_all[col].cat.codes

In [ ]:
plt.figure(figsize=(24,24))
sns.heatmap(df_all.corr(),annot=True, fmt=".2f", square=True)
plt.show()

Heatmapa posłuży przede wszystkim do sprawdzenia, czy do modelu nie używamy dwóch silnie skorelowanych zmiennych. Jednak ciekawie jest też popatrzeć na niektóre zależności i sprawdzać czy zachodzą logiczne związki. 
Na przykład:
- zmienna `Walc` wykazuje dość silny związek z `Dalc` (spożycie alkoholu w tygodniu, ze spożyciem alkoholu w dni powszednie),
- adres ze szkołą - do jednej ze szkół chodziło więcej uczniów z miasta,
- liczba niezaliczeń z chęcią dalszej edukacji,
- czas dojazdu z adresem

Ponadto widzimy, że trzy ostanie wiersze prawie się od siebie nie różnią. Potwierdza się założenie autorów zbioru - nie ma sensu przewidywać na podstawie `G1` i `G2` bo to prawie identyczne zmienne.

Aby lepiej poznać rozkłady poszczególnych zmiennych i ich wpływ na zmienną wyjaśnianą, przygotowaliśmy wykresy skrzypcowe. Ponieważ większość ciekawszych wniosków przedstawiliśmy powyżej, poniżej zostawiliśmy tylko niektóre wykresy

In [ ]:
# fig, axs = plt.subplots(32, 1, figsize=(10, 300))

# for i in range(32):
#     sns.violinplot(data=df, x=df.columns.values[i], y="G3", ax=axs[i])


fig, axs = plt.subplots(9, 1, figsize=(10, 89))
counter=0
for i in [2, 4, 10, 13, 17, 18, 20, 21, 22]:
    sns.violinplot(data=df, x=df.columns.values[i], y="G3", ax=axs[counter])
    counter+=1